# Training and Model Saving Notebook

## 1. Import necessary libraries

In [2]:
#Set up the notebook environment
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import scipy
from scipy.stats import pearsonr
from scipy import signal as sig
from utils import *
import xgboost as xgb
from xgboost import XGBRegressor

## 2. Read Data

In [3]:
raw = scipy.io.loadmat('./datasets/raw_training_data.mat')
data_glove_1 = raw['train_dg'][0][0]
data_glove_1_train = np.delete(data_glove_1, 3, 1)
data_glove_2 = raw['train_dg'][1][0]
data_glove_2_train = np.delete(data_glove_2, 3, 1)
data_glove_3 = raw['train_dg'][2][0]
data_glove_3_train = np.delete(data_glove_3, 3, 1)

ecog_1_train = raw['train_ecog'][0][0]
ecog_2_train = raw['train_ecog'][1][0]
ecog_3_train = raw['train_ecog'][2][0]


raw = scipy.io.loadmat('./datasets/sub1_comp.mat')
ecog_1_comp = raw['train_data']
dg_1_comp = raw['train_dg']
ecog_1_valid = raw['test_data'][49000:]

raw = scipy.io.loadmat('./datasets/sub2_comp.mat')
ecog_2_comp = raw['train_data']
dg_2_comp = raw['train_dg']
ecog_2_valid = raw['test_data'][49000:]

raw = scipy.io.loadmat('./datasets/sub3_comp.mat')
ecog_3_comp = raw['train_data']
dg_3_comp = raw['train_dg']
ecog_3_valid = raw['test_data'][49000:]

dg_1_raw = scipy.io.loadmat('./datasets/sub1_testlabels.mat')
dg_1_valid = dg_1_raw['test_dg'][49000:]
dg_1_valid = np.delete(dg_1_valid, 3, 1)

dg_2_raw = scipy.io.loadmat('./datasets/sub2_testlabels.mat')
dg_2_valid = dg_2_raw['test_dg'][49000:]
dg_2_valid = np.delete(dg_2_valid, 3, 1)

dg_3_raw = scipy.io.loadmat('./datasets/sub3_testlabels.mat')
dg_3_valid = dg_3_raw['test_dg'][49000:]
dg_3_valid = np.delete(dg_3_valid, 3, 1)

In [4]:
winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap

In [5]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_1_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_1_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_1_train
Y_test = dg_1_valid
Y_train = sig.resample(Y_train, feature_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, feature_test.shape[0], axis=0)

R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_1 = feature_selection(R_train, Y_train, 800)
print(idx_1.shape)

R_train = R_train[:, idx_1]
R_test = R_test[:, idx_1]

# create model instance
xgb_reg = XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.01)
# fit model
xgb_reg.fit(R_train, Y_train)
# make predictions
prediction_XGB_1 = xgb_reg.predict(R_test)

print('For Subject 1')
print(f'For XGBoost: {correlation(prediction_XGB_1, Y_test)}')

xgb_reg.save_model('./models/XGB_S1.json')
np.save('./models/idx_S1.npy', idx_1)

/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  warnings.warn('nperseg = {0:d} is greater than input length '


(615,)
For Subject 1
For XGBoost: ([0.5697004646300975, 0.7127210821071518, 0.19972146484178074, 0.3181432431150917], 0.4500715636735304)


In [6]:
winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap

# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_2_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_2_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_2_train
Y_test = dg_2_valid
Y_train = sig.resample(Y_train, feature_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, feature_test.shape[0], axis=0)




R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_2 = feature_selection(R_train, Y_train, 800)
print(idx_2.shape)

R_train = R_train[:, idx_2]
R_test = R_test[:, idx_2]

R_train.shape

/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '


(549,)


(5999, 549)

In [7]:
# create model instance
xgb_reg = XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.01)
# fit model
xgb_reg.fit(R_train, Y_train)
# make predictions
prediction_XGB = xgb_reg.predict(R_test)

print('For Subject 2')
print(f'For XGBoost: {correlation(prediction_XGB, Y_test)}')

xgb_reg.save_model('./models/XGB_S2.json')
np.save('./models/idx_S2.npy', idx_2)

For Subject 2
For XGBoost: ([0.5933382893119988, 0.3820826422678676, 0.2489812591320926, 0.2591780080679073], 0.3708950496949666)


In [8]:
# Compute the R matrix for the training data
feature_train = get_windowed_feats(ecog_3_train, 1000, winLen, winOverlap)
# R_train = create_R_matrix(feature_train, 5)

feature_test = get_windowed_feats(ecog_3_valid, 1000, winLen, winOverlap)
# R_test = create_R_matrix(feature_test, 5)

# Downsample the glove data
Y_train = data_glove_3_train
Y_test = dg_3_valid
Y_train = sig.resample(Y_train, feature_train.shape[0], axis=0)
Y_test = sig.resample(Y_test, feature_test.shape[0], axis=0)

R_train = create_R_matrix(feature_train, 20)
R_test = create_R_matrix(feature_test, 20)

idx_3 = feature_selection(R_train, Y_train, 800)
print(idx_3.shape)

R_train = R_train[:, idx_3]
R_test = R_test[:, idx_3]

# create model instance
xgb_reg = XGBRegressor(n_estimators=200, max_depth=5, learning_rate=0.01)
# fit model
xgb_reg.fit(R_train, Y_train)
# make predictions
prediction_XGB = xgb_reg.predict(R_test)

print('For Subject 3')
print(f'For XGBoost: {correlation(prediction_XGB, Y_test)}')

xgb_reg.save_model('./models/XGB_S3.json')
np.save('./models/idx_S3.npy', idx_3)

/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  warnings.warn('nperseg = {0:d} is greater than input length '


(599,)
For Subject 3
For XGBoost: ([0.7752297742295375, 0.6422236814131032, 0.6144125437786533, 0.6950473981200583], 0.6817283493853381)


In [9]:
ecog_1_leaderboard = ecog_1_comp[500: 500 + 147500]
dg_1_leaderboard = dg_1_comp[500: 500 + 147500]

ecog_2_leaderboard = ecog_2_comp[500: 500 + 147500]
dg_2_leaderboard = dg_2_comp[500: 500 + 147500]

ecog_3_leaderboard = ecog_3_comp[500: 500 + 147500]
dg_3_leaderboard = dg_3_comp[500: 500 + 147500]

winLen = 100 / 1e3
winOverlap = 50 / 1e3
winDisp = winLen - winOverlap


feature_1 = get_windowed_feats(ecog_1_leaderboard, 1000, winLen, winOverlap)
# R_1 = create_R_matrix(feature_1, 5)
feature_2 = get_windowed_feats(ecog_2_leaderboard, 1000, winLen, winOverlap)
# R_2 = create_R_matrix(feature_2, 5)
feature_3 = get_windowed_feats(ecog_3_leaderboard, 1000, winLen, winOverlap)
# R_3 = create_R_matrix(feature_3, 5)

idx_1 = np.load('./models/idx_S1.npy')
idx_2 = np.load('./models/idx_S2.npy')
idx_3 = np.load('./models/idx_S3.npy')

R_1 = create_R_matrix(feature_1, 20)[:, idx_1]
R_2 = create_R_matrix(feature_2, 20)[:, idx_2]
R_3 = create_R_matrix(feature_3, 20)[:, idx_3]


/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 62, using nperseg = 62
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 48, using nperseg = 48
  warnings.warn('nperseg = {0:d} is greater than input length '
/usr/local/lib/python3.11/site-packages/scipy/signal/_spectral_py.py:2014: UserWarning: nperseg = 256 is greater than input length  = 64, using nperseg = 64
  warnings.warn('nperseg = {0:d} is greater than input length '


In [10]:
from xgboost import XGBRegressor
xgb_reg_1 = xgb.XGBRegressor()
xgb_reg_1.load_model("./models/XGB_S1.json")

prediction_1 = xgb_reg_1.predict(R_1)

xgb_reg_2 = xgb.XGBRegressor()
xgb_reg_2.load_model("./models/XGB_S2.json")

prediction_2 = xgb_reg_2.predict(R_2)

xgb_reg_3 = xgb.XGBRegressor()
xgb_reg_3.load_model("./models/XGB_S3.json")

prediction_3 = xgb_reg_3.predict(R_3)

In [11]:
print('For Subject 1')
print(f'For XGBoost: {correlation(sig.resample(prediction_1, dg_1_leaderboard.shape[0]), dg_1_leaderboard)}')

print('For Subject 2')
print(f'For XGBoost: {correlation(sig.resample(prediction_2, dg_2_leaderboard.shape[0]), dg_2_leaderboard)}')

print('For Subject 3')
print(f'For XGBoost: {correlation(sig.resample(prediction_3, dg_3_leaderboard.shape[0]), dg_3_leaderboard)}')

For Subject 1
For XGBoost: ([0.574663377868796, 0.7795334029967799, 0.34396665192992165, 0.3419930322229411], 0.5100391162546096)
For Subject 2
For XGBoost: ([0.6250001985349511, 0.5152723495905296, 0.45129016131573463, 0.41550997017070357], 0.5017681699029797)
For Subject 3
For XGBoost: ([0.6951548666792833, 0.6783205256139645, 0.5508425431322772, 0.5306448577368591], 0.613740698290596)
